<a href="https://colab.research.google.com/github/venuaravind-14/VAC-Value_Added_Course-AI-ML/blob/main/Day4.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle
import numpy as np
import tensorflow

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
with open("/content/train_qa.txt","rb") as fp:
  train_data = pickle.load(fp)

In [ ]:
with open("/content/test_qa.txt","rb") as fp:
  test_data = pickle.load(fp)

In [ ]:
' '.join(train_data[0][0])

'Mary moved to the bathroom . Sandra journeyed to the bedroom .'

In [ ]:
' '.join(train_data[0][1])

'Is Sandra in the hallway ?'

In [ ]:
' '.join(train_data[0][2])

'n o'

In [ ]:
type(test_data)

list

In [ ]:
vocab = set()

In [ ]:
all_data = test_data + train_data

In [ ]:
for story, question, answer in all_data:
  vocab = vocab.union(set(story))
  vocab = vocab.union(set(question))

In [ ]:
vocab.add('no')
vocab.add('yes')

In [ ]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [ ]:
vocab_len=len(vocab)+1

In [ ]:
max_story_len = max([len(data[0]) for data in all_data])

In [ ]:
max_story_len

156

In [ ]:
max_question_len = max([len(data[1]) for data in all_data])

In [ ]:
max_question_len

6

In [ ]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [24]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()

vocab_size = len(tokenizer.word_index) + 1
print("Vocab size:", vocab_size)

Vocab size: 1


In [28]:
import keras

In [29]:
from keras.preprocessing.sequence import pad_sequences

In [30]:
text = ['Mary moved to the bathroom . Sandra journeyed to the bedroom .']

In [31]:
tokenizer = Tokenizer(num_words=18)

In [32]:
tokenizer.fit_on_texts(text)

In [33]:
sequences = tokenizer.texts_to_sequences(text)

In [34]:
# integer encode sequences of words
tokenizer = Tokenizer(filters=[])
tokenizer.fit_on_texts(vocab)

In [35]:
tokenizer.word_index

{'daniel': 1,
 'left': 2,
 'no': 3,
 'down': 4,
 'to': 5,
 'john': 6,
 'yes': 7,
 'moved': 8,
 'hallway': 9,
 'put': 10,
 'went': 11,
 'sandra': 12,
 'office': 13,
 'got': 14,
 'in': 15,
 'took': 16,
 'bedroom': 17,
 'discarded': 18,
 'mary': 19,
 'garden': 20,
 'kitchen': 21,
 'there': 22,
 'apple': 23,
 'milk': 24,
 'journeyed': 25,
 'dropped': 26,
 'grabbed': 27,
 'picked': 28,
 'travelled': 29,
 'back': 30,
 'football': 31,
 'up': 32,
 'the': 33,
 'bathroom': 34,
 'is': 35,
 '?': 36,
 '.': 37}

In [37]:
train_story_text=[]
train_question_text=[]
train_answer=[]

for story, question, answer in train_data:
  train_story_text.append(story)
  train_question_text.append(question)
train_story_seq=tokenizer.texts_to_sequences(train_story_text)
len(train_story_text)

10000

In [38]:
len(train_story_seq)

10000

In [43]:
def vectorize_stories(data, word_index=tokenizer.word_index, max_story_len=max_story_len, max_question_len=max_question_len):
  X = []
  Xq = []
  Y = []
  for story, query, answer in data:
    if '<unk>' not in word_index:
      word_index['<unk>'] = len(word_index) + 1

    x=[word_index.get(word.lower(),word_index['<unk>']) for word in story]
    xq=[word_index.get(word.lower(),word_index['<unk>']) for word in query]
    y=np.zeros(len(word_index)+1)
    y[word_index[answer]]=1

    X.append(x)
    Xq.append(xq)
    Y.append(y)
  return (pad_sequences(X,maxlen=max_story_len),
          pad_sequences(Xq,maxlen=max_question_len),
          np.array(Y))

In [44]:
inputs_train, queries_train, answers_train = vectorize_stories(train_data)

In [45]:
inputs_train.shape

(10000, 156)

In [46]:
queries_train.shape

(10000, 6)

In [47]:
answers_train.shape

(10000, 39)

In [48]:
inputs_test, queries_test, answers_test = vectorize_stories(test_data)